<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import csv
import numpy as np

1. Получение данных

In [2]:
# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/

In [3]:
user_fname = '/home/al/Projects_My/NLP-russian-language/collection_all_resurses_NLP/BERT/classification_ model_for_imbalanced_classes/dataset/creditcard.csv'

In [9]:
all_features = []
all_targets = []
with open(user_fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER: ", line.strip())
            continue # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES: ", all_features[-1])
features = np.array(all_features, dtype = "float32")
targets = np.array(all_targets, dtype = "uint8")

HEADER:  "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES:  [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]


2. Разбиение данных на тренировочные и оценочные.

In [10]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targes = targes[-num_val_samples:]
print("Number of training samples: ", len(train_features))
print("Number of validation samples: ", len(val_features))

Number of training samples:  227846
Number of validation samples:  56961
